# Lab 6: AgentCore 관측성 - Langfuse

## 개요

[Langfuse](https://langfuse.com)는 LLM 애플리케이션을 위한 오픈소스 관측성 플랫폼입니다. Lab 5에서 CloudWatch GenAI Observability를 사용했다면, 이 Lab에서는 Langfuse를 사용하여 더 풍부한 기능을 경험합니다.

### CloudWatch vs Langfuse 비교

| 기능 | CloudWatch GenAI | Langfuse |
|------|------------------|----------|
| **설정** | 자동 (기본값) | 수동 설정 필요 |
| **비용 추적** | 기본 | 상세 모델별 가격 |
| **LLM Playground** | 없음 | 있음 (프롬프트 테스트) |
| **평가 (Evals)** | 수동 | LLM-as-a-judge 내장 |
| **프롬프트 관리** | 없음 | 버전 관리 지원 |
| **오픈소스** | 아니오 | 예 (셀프호스팅 가능) |

### 워크숍 여정

- **Lab 1 (완료)**: 에이전트 프로토타입 생성
- **Lab 2 (완료)**: 메모리로 강화
- **Lab 3 (완료)**: Gateway & Identity로 확장
- **Lab 4 (완료)**: 프로덕션 배포 - AgentCore Runtime
- **Lab 5 (완료)**: CloudWatch GenAI 관측성
- **Lab 6 (현재)**: Langfuse 관측성

### Lab 6의 목표

1. **Langfuse 설정**: API 키 및 OTEL 환경 변수 구성
2. **로컬 테스트**: Langfuse로 트레이스 전송 확인
3. **AgentCore 배포**: Langfuse 환경 변수와 함께 배포
4. **대시보드 탐색**: Langfuse에서 트레이스 분석

### Step 1: 환경 설정

필수 라이브러리를 가져오고 AWS 환경을 설정합니다.

In [ ]:
# 필수 라이브러리 가져오기
import os
import sys
import json
import uuid
import base64
import datetime
import urllib.parse
import requests
import boto3

# 경로 설정
sys.path.insert(0, '../..')

# AWS 리전 확인
boto_session = boto3.session.Session()
region = boto_session.region_name
account_id = boto3.client('sts').get_caller_identity()['Account']

print(f"AWS Region: {region}")
print(f"AWS Account: {account_id}")

### Step 2: Langfuse 자격 증명 설정

Langfuse는 두 가지 방식으로 사용할 수 있습니다:

| 옵션 | 설명 | 장점 |
|------|------|------|
| **Public Langfuse** | Langfuse 클라우드 서비스 사용 | 빠른 설정, 인프라 관리 불필요 |
| **Self-hosted Fargate** | AWS ECS Fargate에 직접 배포 | 데이터 완전 제어, VPC 내부 운영 |

---

#### Option 1: Public Langfuse (권장 - 빠른 시작)

[Langfuse Cloud](https://langfuse.com)를 사용하면 인프라 관리 없이 바로 시작할 수 있습니다.

**설정 방법:**
1. https://cloud.langfuse.com (EU) 또는 https://us.cloud.langfuse.com (US) 접속
2. 계정 생성 및 프로젝트 생성
3. Settings → API Keys에서 키 복사
4. `.env` 파일에 입력:
   ```
   LANGFUSE_PUBLIC_KEY=pk-lf-your-public-key
   LANGFUSE_SECRET_KEY=sk-lf-your-secret-key
   LANGFUSE_BASE_URL=https://us.cloud.langfuse.com
   ```

---

#### Option 2: Self-hosted Langfuse on AWS Fargate

데이터를 AWS 내부에서 관리하려면 Fargate에 Langfuse를 직접 배포할 수 있습니다.

**배포 가이드:** [deploy-langfuse-on-ecs-with-fargate](https://github.com/gonsoomoon-ml/deploy-langfuse-on-ecs-with-fargate)

**설정 방법:**
1. 위 GitHub 저장소의 가이드에 따라 Langfuse 배포
2. 배포 완료 후 ALB 엔드포인트 확인
3. Langfuse 웹 UI에서 프로젝트 생성 및 API 키 발급
4. `.env` 파일에 입력:
   ```
   LANGFUSE_PUBLIC_KEY=pk-lf-your-key
   LANGFUSE_SECRET_KEY=sk-lf-your-key
   LANGFUSE_BASE_URL=http://your-alb-endpoint.region.elb.amazonaws.com
   ```

---

#### .env 파일 생성

```bash
cp .env.example .env
# .env 파일을 열고 선택한 옵션에 맞게 값 입력
```

In [ ]:
# Langfuse 자격 증명 설정
# .env 파일에서 로드 (보안을 위해 키를 코드에 직접 입력하지 않음)
# 
# 설정 방법:
# 1. cp .env.example .env
# 2. .env 파일에 실제 키 입력

from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_BASE_URL = os.getenv("LANGFUSE_BASE_URL", "https://us.cloud.langfuse.com")

# 키 검증
if not LANGFUSE_PUBLIC_KEY or LANGFUSE_PUBLIC_KEY == "pk-lf-your-public-key":
    raise ValueError("⚠️ .env 파일에 LANGFUSE_PUBLIC_KEY를 설정하세요")
if not LANGFUSE_SECRET_KEY or LANGFUSE_SECRET_KEY == "sk-lf-your-secret-key":
    raise ValueError("⚠️ .env 파일에 LANGFUSE_SECRET_KEY를 설정하세요")

# OTEL 인증 헤더 생성 (Basic Auth)
langfuse_auth = base64.b64encode(
    f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()
).decode()

# OTEL 환경 변수 설정
OTEL_ENDPOINT = f"{LANGFUSE_BASE_URL}/api/public/otel"
OTEL_HEADERS = f"Authorization=Basic {langfuse_auth}"

os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = OTEL_ENDPOINT
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = OTEL_HEADERS

print("✅ Langfuse 환경 변수 설정 완료 (.env 파일에서 로드)")
print(f"   Base URL: {LANGFUSE_BASE_URL}")
print(f"   Public Key: {LANGFUSE_PUBLIC_KEY[:20]}...")
print(f"   OTEL Endpoint: {OTEL_ENDPOINT}")

### Step 3: 로컬 테스트 (선택사항)

AgentCore에 배포하기 전에 로컬에서 Langfuse 연동을 테스트합니다.

In [ ]:
# StrandsTelemetry 초기화
from strands.telemetry import StrandsTelemetry

StrandsTelemetry().setup_otlp_exporter()
print("✅ StrandsTelemetry OTEL exporter 설정 완료")

In [ ]:
# 에이전트 생성 및 테스트
from strands import Agent
from strands.models import BedrockModel
from ecommerce_tools import (
    check_return_eligibility,
    process_return_request,
    get_product_recommendations,
    ECOMMERCE_SYSTEM_PROMPT,
    ECOMMERCE_MODEL_ID,
)

model = BedrockModel(model_id=ECOMMERCE_MODEL_ID)

# 테스트용 세션 ID 생성
test_session_id = f"lab6-local-test-{uuid.uuid4().hex[:8]}"

agent = Agent(
    model=model,
    tools=[check_return_eligibility, process_return_request, get_product_recommendations],
    system_prompt=ECOMMERCE_SYSTEM_PROMPT,
    trace_attributes={
        "session.id": test_session_id,
        "user.id": "test-user@example.com",
        "langfuse.tags": ["lab-06", "local-test"],
    },
)

print("✅ 에이전트 생성 완료")
print(f"   Model: {ECOMMERCE_MODEL_ID}")
print(f"   Session ID: {test_session_id}")

In [ ]:
# 에이전트 테스트
test_prompt = "원피스 카테고리에서 인기 상품 추천해주세요."

print(f"📝 질문: {test_prompt}\n")
response = agent(test_prompt)

print("\n📤 응답:")
print("-" * 60)
print(response.message["content"][0]["text"])
print("-" * 60)

print(f"\n🔍 Langfuse에서 확인: {LANGFUSE_BASE_URL}")
print(f"   Session ID로 검색: {test_session_id}")

Local 에서 Langfuse Trace 
![langfuse_local_test_trace.png](../images/langfuse_local_test_trace.png)

### Step 4: AgentCore Runtime 배포 준비

Cognito 및 IAM 설정을 가져옵니다.

In [ ]:
# Secrets Manager에서 Cognito 설정 가져오기
secrets_client = boto3.client('secretsmanager', region_name=region)

secret_name = "ecommerce_customer_support_agent"
secret_response = secrets_client.get_secret_value(SecretId=secret_name)
cognito_config = json.loads(secret_response['SecretString'])

cognito_user_pool_id = cognito_config['pool_id']
cognito_client_id = cognito_config['client_id']
cognito_client_secret = cognito_config['client_secret']
cognito_discovery_url = cognito_config['discovery_url']

print(f"✅ Cognito User Pool ID: {cognito_user_pool_id}")
print(f"✅ Cognito Client ID: {cognito_client_id}")
print(f"✅ Discovery URL: {cognito_discovery_url}")

In [ ]:
# SSM에서 실행 역할 ARN 가져오기
ssm = boto3.client('ssm', region_name=region)

execution_role_param = ssm.get_parameter(Name='/app/ecommerce/agentcore/runtime_iam_role')
execution_role_arn = execution_role_param['Parameter']['Value']

print(f"✅ Execution Role ARN: {execution_role_arn}")

### Step 5: 기존 배포 정리 및 새 배포

기존 에이전트가 있으면 삭제하고 Langfuse 관측성이 활성화된 새 에이전트를 배포합니다.

In [ ]:
import os
import yaml
from bedrock_agentcore_starter_toolkit import Runtime

# 기존 배포가 있으면 삭제
config_file = '.bedrock_agentcore.yaml'
if os.path.exists(config_file):
    print("🔍 기존 배포 확인 중...")
    with open(config_file, 'r') as f:
        existing_config = yaml.safe_load(f)
    
    if existing_config and 'agents' in existing_config:
        for name, cfg in existing_config['agents'].items():
            agent_id = cfg.get('bedrock_agentcore', {}).get('agent_id')
            if agent_id:
                print(f"🗑️ 기존 에이전트 삭제 중: {agent_id}")
                try:
                    client = boto3.client('bedrock-agentcore-control', region_name=region)
                    client.delete_agent_runtime(agentRuntimeId=agent_id)
                    print(f"✅ 에이전트 삭제 완료: {agent_id}")
                except Exception as e:
                    print(f"⚠️ 삭제 실패 (이미 삭제됨?): {e}")
    
    os.remove(config_file)
    print("✅ .bedrock_agentcore.yaml 삭제 완료")

# 에이전트 이름 생성
unique_id = uuid.uuid4().hex[:8]
agent_name = f"lab6_langfuse_{unique_id}"

print(f"\n🚀 에이전트 이름: {agent_name}")
print(f"📁 엔트리포인트: runtime_langfuse_streaming.py")

In [ ]:
# Runtime 설정
runtime = Runtime()

runtime.configure(
    entrypoint="runtime_langfuse_streaming.py",
    execution_role=execution_role_arn,
    agent_name=agent_name,
    region=region,
    # JWT 인증 설정
    authorizer_configuration={
        'customJWTAuthorizer': {
            'allowedClients': [cognito_client_id],
            'discoveryUrl': cognito_discovery_url,
        }
    },
    # ⚠️ AWS ADOT 비활성화 (Langfuse 사용)
    disable_otel=True,
)

print("✅ Runtime 설정 완료")
print(f"   - JWT 인증: 활성화")
print(f"   - AWS ADOT: 비활성화 (disable_otel=True)")
print(f"   - Langfuse: 환경 변수로 전달 예정")

In [ ]:
# 에이전트 배포 (Langfuse 환경 변수 포함)
print("🚀 에이전트 배포 시작...")
print("   (ECR 이미지 빌드 및 AgentCore Runtime 배포)")
print()

# Langfuse 환경 변수와 함께 배포
runtime.launch(
    env_vars={
        "OTEL_EXPORTER_OTLP_ENDPOINT": OTEL_ENDPOINT,
        "OTEL_EXPORTER_OTLP_HEADERS": OTEL_HEADERS,
        "DISABLE_ADOT_OBSERVABILITY": "true",
    }
)

print()
print("✅ 에이전트 배포 완료!")

### Step 6: JWT 토큰 및 세션 설정

In [ ]:
# Bearer Token 획득
from src.helpers.utils import reauthenticate_user

bearer_token = reauthenticate_user(cognito_client_id, cognito_client_secret)

# 관측성 테스트용 세션 생성
# ⚠️ AgentCore는 session_id가 33자 이상이어야 함
session_id = f"lab6-langfuse-session-{uuid.uuid4()}"  # 58자
test_start_time = datetime.datetime.now()

print(f"✅ Bearer Token 획득 완료 (길이: {len(bearer_token)} 문자)")
print()
print("🔍 관측성 테스트 정보")
print("=" * 60)
print(f"세션 ID: {session_id}")
print(f"세션 ID 길이: {len(session_id)}자 (33자 이상 필요)")
print(f"시작 시간: {test_start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)
print("\n💡 위 Session ID로 Langfuse에서 트레이스를 찾을 수 있습니다.")

### Step 7: HTTP + JWT로 에이전트 호출

In [ ]:
# 에이전트 ARN 및 HTTP 엔드포인트 구성
import yaml

with open('.bedrock_agentcore.yaml', 'r') as f:
    agent_config = yaml.safe_load(f)

default_agent = agent_config['default_agent']
agent_arn = agent_config['agents'][default_agent]['bedrock_agentcore']['agent_arn']
agent_id = agent_config['agents'][default_agent]['bedrock_agentcore']['agent_id']

# HTTP 엔드포인트 구성
escaped_arn = urllib.parse.quote(agent_arn, safe='')
invoke_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_arn}/invocations"

# HTTP 요청 헤더
headers = {
    'Authorization': f'Bearer {bearer_token}',
    'Content-Type': 'application/json',
    'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id,
}

print("🌐 HTTP 엔드포인트 정보")
print("=" * 60)
print(f"Agent ID: {agent_id}")
print(f"Agent ARN: {agent_arn}")
print(f"URL: {invoke_url[:80]}...")
print("=" * 60)

In [ ]:
# 첫 번째 호출: 반품 문의
from src.helpers.utils import invoke_agent_http_streaming

test_prompt_1 = "주문번호 ORD-20240115-TEST의 '플라워 패턴 원피스' 반품이 가능한가요?"

response_text_1 = invoke_agent_http_streaming(
    invoke_url=invoke_url,
    headers=headers,
    prompt=test_prompt_1,
    title="테스트 1: 반품 문의",
    extra_payload={
        "session_id": session_id,
        "user_id": "demo-customer@kstyle.com"
    }
)

Langfuse 에서 Trace 확인:
![langfuse_trace_agentcore.png](../images/langfuse_trace_agentcore.png)

In [ ]:
# 두 번째 호출: 상품 추천
test_prompt_2 = "뷰티 카테고리에서 추천 상품 알려주세요."

response_text_2 = invoke_agent_http_streaming(
    invoke_url=invoke_url,
    headers=headers,
    prompt=test_prompt_2,
    title="테스트 2: 상품 추천",
    extra_payload={
        "session_id": session_id,
        "user_id": "demo-customer@kstyle.com"
    }
)

### Step 8: Langfuse 대시보드 확인

Langfuse 대시보드에서 트레이스를 확인합니다.

#### 확인할 수 있는 정보

| 섹션 | 설명 | 활용 |
|------|------|------|
| **Traces** | 모든 에이전트 호출 목록 | 전체 요청 현황 파악 |
| **Sessions** | 세션별 대화 그룹 | 멀티턴 대화 분석 |
| **Timeline** | 각 호출의 상세 타임라인 | 지연 시간 분석 |
| **Scores** | LLM-as-a-judge 평가 | 품질 모니터링 |

In [ ]:
# Langfuse 대시보드 링크
print("🔗 Langfuse 대시보드 링크")
print("=" * 60)
print(f"\n1. Langfuse 대시보드:")
print(f"   {LANGFUSE_BASE_URL}")
print(f"\n2. 트레이스 검색 정보:")
print(f"   세션 ID: {session_id}")
print(f"   사용자 ID: demo-customer@kstyle.com")
print(f"   시작 시간: {test_start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)
print("\n💡 Langfuse에서 확인할 수 있는 정보:")
print("   - 각 LLM 호출의 입력/출력 토큰 수")
print("   - 도구 호출 내역 및 결과")
print("   - 전체 비용 및 지연 시간")
print("   - 세션별 대화 그룹화")

### Langfuse 트레이스 구조

위의 두 호출은 Langfuse에서 다음과 같이 기록됩니다:

```
Session: lab6-langfuse-session-xxx
├── Trace 1: 반품 문의
│   ├── Generation: llm.chat (1st LLM call)
│   │   ├── Input: system prompt + user question
│   │   ├── Output: tool_use decision
│   │   └── Usage: input_tokens, output_tokens
│   ├── Span: tool.check_return_eligibility
│   │   ├── Input: {order_id, item_name}
│   │   └── Output: return policy info
│   └── Generation: llm.chat (2nd LLM call)
│       ├── Input: previous context + tool result
│       ├── Output: final response
│       └── Usage: input_tokens, output_tokens
│
└── Trace 2: 상품 추천
    └── ... (유사한 구조)
```

### Step 9: 정리 (선택사항)

테스트가 완료되면 배포된 에이전트를 삭제할 수 있습니다.

In [ ]:
# 에이전트 삭제 (선택사항)
# 아래 주석을 해제하고 실행하면 배포된 에이전트가 삭제됩니다.

import yaml                                                                                                
                                                                                                            
config_file = '.bedrock_agentcore.yaml'                                                                    
if os.path.exists(config_file):                                                                            
    with open(config_file, 'r') as f:                                                                      
        agent_config = yaml.safe_load(f)                                                                   
                                                                                                            
    default_agent = agent_config['default_agent']                                                          
    agent_id = agent_config['agents'][default_agent]['bedrock_agentcore']['agent_id']                      
                                                                                                            
    print("🗑️ 에이전트 삭제 중...")                                                                         
    client = boto3.client('bedrock-agentcore-control', region_name=region)                                 
    client.delete_agent_runtime(agentRuntimeId=agent_id)                                                   
    print(f"✅ 에이전트가 삭제되었습니다: {agent_id}")                                                     
                                                                                                            
    os.remove(config_file)                                                                                 
    print("✅ .bedrock_agentcore.yaml 삭제 완료")                                                          
else:                                                                                                      
    print("⚠️ 배포된 에이전트가 없습니다.")   

### 🎉 축하합니다!

**Lab 6: AgentCore Langfuse 관측성**을 성공적으로 완료했습니다!

#### 달성한 성과:

##### Langfuse 설정:
- Langfuse API 키 및 OTEL 환경 변수 구성
- StrandsTelemetry를 통한 자동 트레이스 수집

##### AgentCore 배포:
- `disable_otel=True`로 AWS ADOT 비활성화
- `env_vars`로 Langfuse 환경 변수 전달

##### 관측성 기능:
- 세션별 대화 그룹화
- LLM 호출 및 도구 사용 추적
- 토큰 사용량 및 비용 모니터링

#### CloudWatch vs Langfuse 요약:

| 항목 | Lab 5 (CloudWatch) | Lab 6 (Langfuse) |
|------|-------------------|------------------|
| 설정 | 자동 | `env_vars`로 전달 |
| OTEL | AWS ADOT | StrandsTelemetry |
| Dockerfile | `opentelemetry-instrument` | 직접 실행 |
| 대시보드 | CloudWatch GenAI | Langfuse |
| 추가 기능 | X-Ray 연동 | LLM Playground, Evals |